In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=50,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

vectorStore = Chroma.from_documents(docs, cached_embedding)

retriver = vectorStore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, dont't makt it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = {"context":retriver} | prompt | llm

chain.invoke(question = "Describe Victory Mansions")

VectorStoreRetriever(tags=['Chroma', 'CacheBackedEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x000002D9C29AC610>)